# CSC 480-F25 Lab 2: Task Decomposition, Multi‑Agent Design Patterns, and Communication Protocols

# Authors:

***Arnav Bhola, Pranav Krishna***

California Polytechnic State University, San Luis Obispo;

Computer Science & Software Engineering Department

# Overview

This lab focuses on:
- Breaking down complex problems into smaller tasks (task decomposition)
- Defining specialized agents with clear roles and responsibilities  
- Choosing appropriate multi-agent design patterns
- Specifying communication protocols between agents
- Implementing a three-agent system using AutoGen

NOTE: If you already know what you're going to do for your final project, you may use this lab to make progress on it. Otherwise, treat this lab as a distinct exercise.

## Learning Objectives

By the end of this lab, you will be able to:

- Decompose a complex problem into a comprehensive set of constituent tasks
- Identify and define the roles, responsibilities, and abilities of individual agents
- Select and justify an appropriate multi-agent design pattern (Manager‑Worker, Sequential Pipeline, Collaborative Team)
- Explain and apply foundational agent communication protocols at a high level (MCP, A2A)
- Start thinking about the architecture of your project (agents, tasks, design pattern, and interaction protocols)

# Part 1: Task Decomposition and Agent Design

## 1. Problem Statement & Task Decomposition

**Main Problem:** This agent will solve the issue of bug fixing in code files

**Task Breakdown:** List the steps and sub-tasks needed to achieve the goal:
1. This agent will first find the code block(s) that are associated with the bug
2. This agent will patch/fix the bug and provide the new code 
3. This agent will check the code to ensure the bug is fixed and no new errors are created by this fix. If a new error is created, it will repeat step 3.

Agent Exits once code is working.

## 2. Agent Definition

Define your primary agents with clear roles and responsibilities:

### Agent 1: Bug Context Identifier Agent
- **Role:** Worker Agent (Finds code chunks related to bug)
- **Responsibilities:** Finds the code chunks related to the bug in context with the whole program
- **Inputs:** Gets user input for the code as well as the description of the bug
- **Outputs:** Returns the code chunks related to bug
- **Success Criteria:** Successful if the agent returns a code chunk, if no code chunk returned, then bug doesn't exist

### Agent 2: Code Fixer Agent 
- **Role:** Worker Agent (Fixes the code chunk to patch the bug)
- **Responsibilities:** Fixes the code chunk to patch the bug
- **Inputs:** Recieves code chunks from Agent 1
- **Outputs:** Outputs the updated/fixed code of the whole program
- **Success Criteria:** Successful, if the agent returns the fixed code of the whole program, failed if it doesn't output any code or other unneeded content.

### Agent 3: QA Agent
- **Role:** Manager Agent (Quality checks the code)
- **Responsibilities:** Quality checks the code and if any errors are found, it will return a description of the bug and provide the code chunk back to agent 2.
- **Inputs:** Recieves the full code of the "fixed" program
- **Outputs:** Returns the code of the whole program
- **Success Criteria:** Successful, if the agent returns the fixed code of the whole program, failed if it doesn't output any code or other unneeded content.

## 3. Design Pattern Selection

**Chosen Pattern:** Sequential Pipeline

**Justification:** This pattern works since this follows a pipeline-like approach where it follows a series of steps to fix the code, and at the end there could be a cyclic step which repeats until errors are caught and fixed.



## 4. Communication Design

### Model Context Protocol (MCP)
**Shared Context & Tools:** Full code base from the user input

### Agent-to-Agent (A2A) Interactions

#### Interaction 1: [User] → [Agent 1]
- **Purpose:** This sends the full program code + bug description to Agent 1
- **Key Fields:** Full code + description of bug 
- **Message Format:** Message structure consists of the full program code and a description of the bug

#### Interaction 2: [Agent 1] → [Agent 2]
- **Purpose:** This sends the output of Agent 1 (the detected code blocks related to the bug) to Agent 2
- **Key Fields:** Code field + bug description message
- **Message Format:** Message structure consists of a code field and a description of the bug

#### Interaction 3: [Agent 2] → [Agent 3] 
- **Purpose:** This sends the fixed code of the whole program to the QA agent
- **Key Fields:** Full code + description of bug
- **Message Format:** Full program code and a description of the bug

#### Interaction 4: [Agent 3] → [Agent 2]  (OPTIONAL)
- **Purpose:** If issues are found by QA agent, this will send any new bugs/issues detected to Agent 2 to fix
- **Key Fields:** Full code + description of bug
- **Message Format:** Full program code and a description of the bug

### Exit


## 5. Interaction Diagram
```
                   ←    ←    ←
                   ↓         ↑
User → Agent 1 → Agent 2 → Agent 3 → Exit
  ↓      ↓         ↓         ↓
       A2A        A2A       A2A
Msg1   Msg2       Msg3      Msg4
```

# Part 2: Three-Agent AutoGen Implementation

## Environment Setup

Install required packages and set up Azure OpenAI configuration.

In [13]:
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [15]:
%pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
from pathlib import Path
from dotenv import load_dotenv

In [17]:
cwd = Path.cwd()
env_path = cwd.parent / ".env"
loaded = load_dotenv(env_path)

In [18]:
# This is the same as in L1
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = "2024-12-01-preview"  # If you're using gpt-5-mini as demonstrated

# e.g. https://ccoha-mfoynknp-eastus2.cognitiveservices.azure.com/
azure_endpoint = os.getenv("AZURE_ENDPOINT")

## An Example Three-Agent System Architecture

Based on the suggested roles from the overview:
- **Planner**: Decomposes the user goal into a concise, ordered plan
- **Implementer**: Executes the steps from the plan
- **Critic/Integrator**: Reviews output and suggests improvements or approves completion

In [19]:
async def setup_three_agent_system():
    """Set up the three-agent system with Planner, Implementer, and Critic roles"""
    
    # Create the Azure OpenAI client
    client = AzureOpenAIChatCompletionClient(
        azure_deployment=azure_deployment,
        model="gpt-5-mini",
        api_version=api_version,
        azure_endpoint=azure_endpoint,
        api_key=os.getenv("AZURE_SUBSCRIPTION_KEY"),
    )
    
    # Define the three agents with specific system prompts
    planner = AssistantAgent(
        name="Planner",
        model_client=client,
        system_message="""You are a task decomposition specialist. Your role is to:
        1. Break down complex user goals into clear, numbered steps
        2. Define specific inputs/outputs for each step  
        3. Provide testable acceptance criteria for each step
        4. Present plans in a structured, actionable format
        
        Always end your response with 'PLAN_COMPLETE' when finished."""
    )
    
    implementer = AssistantAgent(
        name="Implementer", 
        model_client=client,
        system_message="""You are an implementation specialist. Your role is to:
        1. Follow the Planner's steps methodically
        2. Address each step with concrete outputs/artifacts
        3. Provide detailed solutions that meet the specified criteria
        4. Ask for clarification if any step is unclear
        
        Always end your response with 'IMPLEMENTATION_COMPLETE' when finished."""
    )
    
    critic = AssistantAgent(
        name="Critic",
        model_client=client, 
        system_message="""You are a quality assurance specialist. Your role is to:
        1. Review the Implementer's work against the Planner's criteria
        2. Identify gaps, errors, or missing elements
        3. Suggest specific improvements if needed
        4. Only approve with 'APPROVED' if all criteria are fully met
        
        Be thorough but constructive in your feedback."""
    )
    
    return planner, implementer, critic

In [20]:
async def run_three_agent_workflow(task_description):
    """Run the three-agent workflow for a given task"""

    # Set up the agents
    planner, implementer, critic = await setup_three_agent_system()

    # Create termination condition - look for "APPROVED" in messages
    # See https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TerminationCondition
    termination = TextMentionTermination("APPROVED")

    # Create a group chat with the three agents
    # The RoundRobinGroupChat will manage turn-taking between agents
    # See https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat
    team = RoundRobinGroupChat([planner, implementer, critic], termination_condition=termination)

    # Run the workflow
    print("Starting three-agent workflow...")
    print("=" * 60)

    # Start the conversation
    result = await Console(
        team.run_stream(task=task_description)
    )

    return result

## Example Task 1: Analytical Task

Test the three-agent system with a small example analytical task.

In [21]:
# Test Task 1: Compare sorting algorithms
analytical_task = """
Compare the pros and cons of Quick Sort vs Merge Sort algorithms. 
Provide a structured analysis that includes:
- Time and space complexity for both
- Best/worst case scenarios  
- Practical use cases for each
- A final recommendation with justification
"""

# Uncomment to run when ready:
await run_three_agent_workflow(analytical_task)

Starting three-agent workflow...
---------- TextMessage (user) ----------

Compare the pros and cons of Quick Sort vs Merge Sort algorithms. 
Provide a structured analysis that includes:
- Time and space complexity for both
- Best/worst case scenarios  
- Practical use cases for each
- A final recommendation with justification

---------- TextMessage (Planner) ----------
1) Step 1 — Define the algorithms and key characteristics
- Input: request to compare Quick Sort and Merge Sort.
- Output: concise definitions and core characteristics (in-place/stable/partitioning/merge).
- Acceptance criteria (testable):
  - The output names both algorithms and gives a one-sentence description for each.
  - The output states whether each algorithm is typically in-place and whether it is stable.

Deliverable:
1.1 Quick Sort: A divide-and-conquer sorting algorithm that partitions an array around a pivot, recursively sorts partitions. Typically in-place (no large auxiliary arrays) and not stable in its 

TaskResult(messages=[TextMessage(id='fc2bf458-0e06-4c94-8568-b8945410da32', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 1, 4, 21, 8, 815245, tzinfo=datetime.timezone.utc), content='\nCompare the pros and cons of Quick Sort vs Merge Sort algorithms. \nProvide a structured analysis that includes:\n- Time and space complexity for both\n- Best/worst case scenarios  \n- Practical use cases for each\n- A final recommendation with justification\n', type='TextMessage'), TextMessage(id='a291d7f9-87b6-410a-bd80-8b15e466c160', source='Planner', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=2468), metadata={}, created_at=datetime.datetime(2025, 10, 1, 4, 21, 45, 210451, tzinfo=datetime.timezone.utc), content='1) Step 1 — Define the algorithms and key characteristics\n- Input: request to compare Quick Sort and Merge Sort.\n- Output: concise definitions and core characteristics (in-place/stable/partitioning/merge).\n- Acceptance criteria (te

## Example Task 2: Creative Task

Test the three-agent system with a creative writing task.

In [22]:
# Test Task 2: Creative writing with constraints  
creative_task = """
Write a 150-word summary of the benefits of renewable energy that:
- Uses exactly 3 specific statistics or data points
- Includes at least 2 types of renewable energy sources
- Has a compelling call-to-action in the final sentence
- Maintains an optimistic but factual tone throughout
"""

# Uncomment to run when ready:
await run_three_agent_workflow(creative_task)

Starting three-agent workflow...
---------- TextMessage (user) ----------

Write a 150-word summary of the benefits of renewable energy that:
- Uses exactly 3 specific statistics or data points
- Includes at least 2 types of renewable energy sources
- Has a compelling call-to-action in the final sentence
- Maintains an optimistic but factual tone throughout

---------- TextMessage (Planner) ----------
1) Identify and formalize constraints
- Inputs: user prompt listing constraints (word count, number of statistics, required renewable types, final-sentence CTA, tone)
- Outputs: explicit checklist of constraints to satisfy
- Acceptance criteria (testable): checklist includes: 150-word target; exactly 3 numeric statistics; at least 2 renewable types named; final sentence is a clear call-to-action; tone is optimistic but factual

2) Select three factual, concise data points
- Inputs: reliable background knowledge; constraint to avoid extra numerals in the summary
- Outputs: chosen statistic

TaskResult(messages=[TextMessage(id='d18e7336-486d-4bc5-86e4-76452f81386d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 1, 4, 22, 28, 824865, tzinfo=datetime.timezone.utc), content='\nWrite a 150-word summary of the benefits of renewable energy that:\n- Uses exactly 3 specific statistics or data points\n- Includes at least 2 types of renewable energy sources\n- Has a compelling call-to-action in the final sentence\n- Maintains an optimistic but factual tone throughout\n', type='TextMessage'), TextMessage(id='3f180ad4-19e2-4123-a4ba-a3db9c8c742a', source='Planner', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=3735), metadata={}, created_at=datetime.datetime(2025, 10, 1, 4, 23, 8, 104653, tzinfo=datetime.timezone.utc), content='1) Identify and formalize constraints\n- Inputs: user prompt listing constraints (word count, number of statistics, required renewable types, final-sentence CTA, tone)\n- Outputs: explicit checklist of co

## Your Task

Write your own task for the planner-implementer-critic system

In [23]:
your_task = """
Create a 5 day travel itinerary to a random country of your choice. This itinerary should:
- Include at least 2 activities per day
- Be realistic in terms of travel time and logistics
- Include a mix of cultural, historical, and leisure activities
- The 4th night should include a special dining experience
"""

# Uncomment to run when ready:
await run_three_agent_workflow(your_task)

Starting three-agent workflow...
---------- TextMessage (user) ----------

Create a 5 day travel itinerary to a random country of your choice. This itinerary should:
- Include at least 2 activities per day
- Be realistic in terms of travel time and logistics
- Include a mix of cultural, historical, and leisure activities
- The 4th night should include a special dining experience



---------- TextMessage (Planner) ----------
Random country chosen: Portugal — base: Lisbon with day trips to Sintra and Cascais. Below is a clear, actionable 5-day itinerary presented as numbered steps (Day 0 = pre-trip preparation + logistics), with specific inputs/outputs and testable acceptance criteria for each step.

Step 0 — Pre-trip preparation (do this 2–14 days before travel)
1) Activities
   - Book round-trip flights to Lisbon (LIS).
   - Reserve 4 nights accommodation in central Lisbon (Baixa/Chiado or Alfama recommended).
   - Pre-book timed tickets for Pena Palace and Quinta da Regaleira (Sintra) and one museum in Lisbon if possible (MAAT or National Tile Museum).
   - Make a dinner reservation for Day 4’s special dining experience (e.g., Belcanto — reserve at least 2–3 weeks ahead).
   - Purchase or plan local transport: Viva Viagem card for tram/metro/bus and note train times for Sintra/Cascais day trips.
2) Inputs
   - Passport/ID, flight confirmation, hotel confirmatio

TaskResult(messages=[TextMessage(id='772a2bd0-f524-4e97-b2da-1c6e6b34d280', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 1, 4, 23, 51, 349253, tzinfo=datetime.timezone.utc), content='\nCreate a 5 day travel itinerary to a random country of your choice. This itinerary should:\n- Include at least 2 activities per day\n- Be realistic in terms of travel time and logistics\n- Include a mix of cultural, historical, and leisure activities\n- The 4th night should include a special dining experience\n', type='TextMessage'), TextMessage(id='20460e9b-d513-49a2-967a-777408c6f7a4', source='Planner', models_usage=RequestUsage(prompt_tokens=157, completion_tokens=3146), metadata={}, created_at=datetime.datetime(2025, 10, 1, 4, 24, 18, 242532, tzinfo=datetime.timezone.utc), content='Random country chosen: Portugal — base: Lisbon with day trips to Sintra and Cascais. Below is a clear, actionable 5-day itinerary presented as numbered steps (Day 0 = pre-trip prepa

NOTE: You'll notice that the output might contain duplicated text, that is because `Console` prints when a text message is sent and received.

## Reflection & Analysis

### What worked well?
The agent system was quite good at providing detailed, coherent responses. The responses were generally in depth and thorough, covering required aspects and sometimes going above and beyond. The three pronged approach to answering a query/accomplishing a task worked well because it didn't overwhelm any singular agent, but instead allowed for each agent to execute its relative task to the fullest and produce a proper response. 

### What struggled?
The system struggled with following specific guidelines at times. For instance, it did not meet the 150 word requirement for the second task. Also, sometimes it did more than asked and went overboard with its responses, which could be both a good and bad thing.

### Potential improvements:
One improvement could be providing some more distinction between the tasks/outcomes of the planning agent and the implementer agent. Sometimes the implementer agent would simply take the outputs of the planning agent and restructure the response, and would not add its own additions. Ensuring the planning agent only does planning might lead to a more efficient system.

### Tool integration:
[Optional: Discuss how adding tools (like a calculator function) might improve the system]

# Optional Extensions

## Adding Tools (Optional Challenge)

Consider adding a simple tool function that agents can use, such as a calculator or text analyzer.

In [24]:
# Example tool function (uncomment and modify as needed)
# def calculator(expression: str) -> float:
#     """Simple calculator tool for basic mathematical operations"""
#     try:
#         # WARNING: eval() should not be used in production - use a proper math parser
#         result = eval(expression)
#         return result
#     except Exception as e:
#         return f"Error: {str(e)}"

# To integrate tools with AutoGen agents, you would typically:
# 1. Define tool schemas following MCP-like patterns
# 2. Register tools with specific agents  
# 3. Update system prompts to mention available tools
# 4. Handle tool calls in the conversation flow

# Summary and Next Steps

## Key Takeaways

- **Task Decomposition**: We learned that breaking down complex problems seems difficult at first, but when you clearly lay out the scope and take a look at the big picture, you are able to section out and identify independent tasks. You can then delegate these to, in our case, various agents that can come together to solve the problem.
- **Agent Design**: We learned that when it comes to defining agent roles and responsibilities, it is important too never delegate too much or even too little to one agent. If you make an agent's task too complex, it may not be able to perform it effetively and that might indicate adding another agent. But if you provide the agent with a simple ask, it begs the question of whether an agent is needed in the first place. Ultimately, when building agents, the goal/output of the agent must be reasonable and clearly defined.
- **Communication Protocols**: MCP is a standardized protocol used by agents to gain access to a series of tools, resources, and prompts. It facilitates agent->tool interactions. A2A on the other hand is a standardized agent->agent communication protocol. Agents built on different frameworks can communicate with easy. Our agent system in part 1 used various A2A interactions, as well having the user's codebase as a shared context / potential MCP tool.

## References

- [AutoGen Documentation](https://microsoft.github.io/autogen/stable/index.html)
- [Model Context Protocol](https://modelcontextprotocol.io/docs/getting-started/intro)
- [Agent-to-Agent Protocol](https://a2a-protocol.org/latest/)
- L2_overview.md (Lab requirements and detailed explanations)